In [1]:
from openai import OpenAI
import os
import logging
from jinja2 import Environment, select_autoescape
import json
from miniautogen.pipeline.pipeline import PipelineComponent
from miniautogen.pipeline.components import (
    NextAgentSelectorComponent,
)

class Jinja2SingleTemplateComponent(PipelineComponent):
    def __init__(self):
        """
        Inicializa o componente sem um template ou variáveis.
        """
        self.template_str = None
        self.variables = None
        self.env = Environment(autoescape=select_autoescape())

    
    def set_template_str(self, template_str):
        """
        Configura a string do template para o componente.

        Args:
            template_str (str): String contendo o template Jinja2.
        """
        self.template_str = template_str
    
    def set_variables(self, variables):
        """
        Configura as variáveis para o componente.

        Args:
            variables (dict): Dicionário de variáveis para o template.
        """
        self.variables = variables

    def process(self, state):
        """
        Processa o estado atual do pipeline, substituindo as variáveis no template.

        Args:
            state (PipelineState): Estado atual do pipeline contendo as variáveis.

        Returns:
            PipelineState: Estado atualizado do pipeline.
        """
        template = self.env.from_string(self.template_str)
        chat = state.get_state().get('group_chat')
        agent = state.get_state().get('selected_agent')
        messages = json.loads(chat.get_messages()[['sender_id', 'message']].to_json(orient='records'))

        # Verifica se as variáveis foram definidas
        if self.variables is None:
            self.variables = state.get_state().get('variables', {})

        self.variables['chat'] = chat
        self.variables['agent'] = agent
        self.variables['messages'] = messages
        # Renderiza o template com as variáveis fornecidas
        prompt = template.render(self.variables)
        prompt = json.loads(prompt)
        print(prompt)

        # Atualiza o estado do pipeline com a saída renderizada
        state.update_state(prompt=prompt)

        return state
    

class OpenAIComponent(PipelineComponent):
    """
    Componente de Pipeline para gerar respostas utilizando o modelo de linguagem da OpenAI.
    """

    def __init__(self):
        self.client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
        self.logger = logging.getLogger(__name__)

    def process(self, state):
        try:
            prompt = state.get_state().get('prompt')
            if not prompt:
                raise ValueError(
                    "groupchat e agent são obrigatórios para OpenAIResponseComponent.")
            response = self._call_openai_api(prompt)
            return response.choices[0].message.content
        except Exception as e:
            self.logger.error(f"Erro em OpenAIResponseComponent: {e}")
            raise

    def _call_openai_api(self, prompt):
        """ Realiza a chamada à API da OpenAI. """
        try:
            return self.client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=prompt,
                temperature=1
            )
        except Exception as e:
            self.logger.error(f"Erro ao chamar a API da OpenAI: {e}")
            raise


class NextAgentMessageComponent(PipelineComponent):
    def __init__(self):
        self.alternative_next = NextAgentSelectorComponent()
    
    def set_alternative_next(self, alternative_next):
        """
        Configura o próximo componente a ser executado caso não seja encontrado um agente
        na última mensagem.

        Args:
            alternative_next (PipelineComponent): Próximo componente a ser executado.
        """
        self.alternative_next = alternative_next

    def process(self, state):
        """
        Processa a seleção do próximo agente com base na última mensagem do chat.

        Args:
            state (PipelineState): Estado atual do pipeline.

        Returns:
            PipelineState: Estado atualizado do pipeline.
        """
        chat = state.get_state().get('group_chat')
        agents = chat.agentList

        # Obtém a última mensagem do chat
        messages = chat.get_messages()
        last_message = messages.iloc[-1].message if not messages.empty else None

        next_agent = None
        if last_message:
            # Procura pelo agent_id de cada agente na última mensagem
            for agent in agents:
                if agent.agent_id in last_message:
                    next_agent = agent
                    break

        # Atualiza o estado com o próximo agente selecionado, se encontrado
        if next_agent:
            state.update_state(selected_agent=next_agent)
        else:
            print("Nenhum agente correspondente encontrado na última mensagem.")
            self.alternative_next.process(state)

        return state
    

class UpdateNextAgentComponent(PipelineComponent):
    def __init__(self):
        """
        Inicializa o componente com a lista de agentes disponíveis.

        Args:
            agents (list): Lista de agentes disponíveis no sistema.
        """
        self.next_agent_id = None
    
    def set_next_agent_id(self, next_agent_id):
        """
        Configura o ID do próximo agente a ser definido no estado.

        Args:
            next_agent_id (str): ID do próximo agente.
        """
        self.next_agent_id = next_agent_id

    def process(self, state):
        """
        Atualiza o estado para indicar o próximo agente com base no agent_id fornecido.

        Args:
            state (PipelineState): Estado atual do pipeline.
            agent_id (str): ID do agente a ser definido como o próximo.

        Returns:
            PipelineState: Estado atualizado do pipeline.
        """
        chat = state.get_state().get('group_chat')
        agents = chat.agentList

        for agent in agents:
            if agent.agent_id in self.next_agent_id:
                next_agent = agent
                break
        
        if next_agent:
            state.update_state(selected_agent=next_agent)
            print(f"Próximo agente atualizado para: {self.next_agent_id}")
        else:
            raise ValueError(f"Agent ID '{self.next_agent_id}' não encontrado entre os agentes disponíveis.")

        return state



In [2]:
from miniautogen.chat.chat import Chat
from miniautogen.agent.agent import Agent
from miniautogen.chat.chatadmin import ChatAdmin
from miniautogen.pipeline.pipeline import Pipeline
from miniautogen.pipeline.components import (
    UserResponseComponent,
    AgentReplyComponent,
    TerminateChatComponent
)

template_str = """
[
  {"role": "system", "content": "{{ agent.role }}"}{% for message in messages %},
  {% if message.sender_id == agent.agent_id %}
    {"role": "assistant", "content": {{ message.message | tojson | safe }}}
  {% else %}
    {"role": "user", "content": {{ message.message | tojson | safe }}}
  {% endif %}
{% endfor %}
]
"""

# Cria uma instância do componente com o template
jinja_component = Jinja2SingleTemplateComponent()
jinja_component.set_template_str(template_str)

UpdateNextAgent = UpdateNextAgentComponent()
UpdateNextAgent.set_next_agent_id("Bruno")

NextAgentMessage = NextAgentMessageComponent()
NextAgentMessage.set_alternative_next(UpdateNextAgent)

# Configuração dos Pipelines
pipeline_user = Pipeline([UserResponseComponent()])
pipeline_jinja = Pipeline([jinja_component, OpenAIComponent()])
pipeline_admin = Pipeline(
    [NextAgentMessage, AgentReplyComponent(), TerminateChatComponent()])

# Setup do ambiente de teste
chat = Chat()
json_data = {
    'agent_id': 'Bruno',
    'name': 'Bruno',
    'role': 'user'
}

# Criação de Agentes
agent1 = Agent.from_json(json_data)
agent1.pipeline = pipeline_user  # Atribuindo o pipeline ao agente

agent2 = Agent("dev", "Carlos", "Python senior Developer")
agent2.pipeline = pipeline_jinja  # Atribuindo o pipeline_llm ao segundo agente


# Adicionando os agentes ao chat
chat.add_agent(agent1)
chat.add_agent(agent2)

# Adição de mensagens de teste ao chat
json_messages = [
    {'sender_id': 'user4', 'message': 'Que bom ouvir isso!'},
    {'sender_id': 'user5', 'message': 'Vamos continuar Bruno.',
        'additional_info': {'topic': 'chat'}},
]
chat.add_messages(json_messages)

# Criação e configuração do ChatAdmin
chat_admin = ChatAdmin("admin", "Admin", "admin_role",
                       pipeline_admin, chat, "manage_chat", 10)

In [3]:
chat_admin.run()

INFO:miniautogen.chat.chatadmin:Chat Admin started.
INFO:miniautogen.chat.chatadmin:Executing round 1
INFO:miniautogen.chat.chatadmin:Executing round 2


dev, o que voce acha sobre isso?
[{'role': 'system', 'content': 'Python senior Developer'}, {'role': 'user', 'content': 'Defesa, o que acha?'}, {'role': 'user', 'content': 'Que bom ouvir isso!'}, {'role': 'user', 'content': 'Vamos continuar Bruno.'}, {'role': 'user', 'content': ''}, {'role': 'user', 'content': 'Defesa, o que vc acha?'}, {'role': 'user', 'content': 'Defesa'}, {'role': 'user', 'content': 'terminate'}, {'role': 'user', 'content': 'Defesa, o que acha?'}, {'role': 'user', 'content': 'Que bom ouvir isso!'}, {'role': 'user', 'content': 'Vamos continuar Bruno.'}, {'role': 'user', 'content': 'dev, o que voce acha sobre isso?'}]


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:miniautogen.chat.chatadmin:Executing round 3


Como sou um assistente de IA, não tenho opinião. No entanto, posso fornecer informações ou sugestões com base em dados e padrões previamente estabelecidos. Se você me fornecer mais contexto ou detalhes específicos sobre o assunto em questão, ficarei feliz em ajudar.
Nenhum agente correspondente encontrado na última mensagem.
Próximo agente atualizado para: Bruno


INFO:miniautogen.chat.chatadmin:Chat Admin stopped.
INFO:miniautogen.chat.chatadmin:Chat Admin stopped.


terminate
Encerrando chat...


In [4]:
chat.get_messages().to_dict(orient='records')

[{'id': 1,
  'sender_id': 'agent_admin',
  'message': 'Defesa, o que acha?',
  'timestamp': Timestamp('2023-12-29 10:20:37.244032'),
  'additional_info': None},
 {'id': 2,
  'sender_id': 'user4',
  'message': 'Que bom ouvir isso!',
  'timestamp': Timestamp('2023-12-29 10:31:11.848860'),
  'additional_info': None},
 {'id': 3,
  'sender_id': 'user5',
  'message': 'Vamos continuar Bruno.',
  'timestamp': Timestamp('2023-12-29 10:31:11.850744'),
  'additional_info': {'topic': 'chat'}},
 {'id': 4,
  'sender_id': 'Bruno',
  'message': '',
  'timestamp': Timestamp('2023-12-29 10:31:11.989483'),
  'additional_info': None},
 {'id': 5,
  'sender_id': 'Bruno',
  'message': 'Defesa, o que vc acha?',
  'timestamp': Timestamp('2023-12-29 10:31:17.938861'),
  'additional_info': None},
 {'id': 6,
  'sender_id': 'Bruno',
  'message': 'Defesa',
  'timestamp': Timestamp('2023-12-29 10:31:29.099221'),
  'additional_info': None},
 {'id': 7,
  'sender_id': 'Bruno',
  'message': 'terminate',
  'timestamp': T